In [8]:
import datetime
import plaid
from flask import request
from flask import Flask
import json
import pandas as pd

In [9]:
import sys
sys.path.insert(1, './private')
import keys as keys

In [10]:
'{:%Y-%m-%d}'.format(datetime.datetime.now())

'2019-08-14'

In [11]:
client = plaid.Client(client_id = keys.PLAID_CLIENT_ID, secret=keys.PLAID_SECRET,
                      public_key=keys.PLAID_PUBLIC_KEY, environment=keys.PLAID_ENV, api_version='2019-05-29')


/Users/kzahir/anaconda3/lib/python3.6/site-packages/plaid/client.py:66: UserWarning: 
                Development is not intended for production usage.
                Swap out url for https://production.plaid.com
                via Client.config before switching to production
            
  ''')


In [37]:
app = Flask(__name__)

@app.route('/get_access_token', methods=['POST'])
def get_access_token():
  global access_token
  public_token = request.form['public_token']
  try:
    exchange_response = client.Item.public_token.exchange(public_token)
  except plaid.errors.PlaidError as e:
    return jsonify(format_error(e))

  pretty_print_response(exchange_response)
  access_token = exchange_response['access_token']
  return jsonify(exchange_response)

start_date = '{:%Y-%m-%d}'.format(datetime.datetime.now() + datetime.timedelta(-30))
end_date = '{:%Y-%m-%d}'.format(datetime.datetime.now())
transactions_response = client.Transactions.get(access_token,
                                   start_date=start_date,
                                   end_date=end_date, count=500)
accountList= pd.DataFrame(columns=['account', 'subtype', 'type', 'balanceA', 'balanceC' , 'accountId'])
for account in transactions_response['accounts']:
    name = account['name']
    subtype = account['subtype']
    type = account['type']
    balanceA= account['balances']['available']
    balanceC= account['balances']['current']
    accountId= account['account_id']
    
    b = pd.Series({'account':name,'subtype':subtype,'type':type, 'balanceA':balanceA, 'balanceC':balanceC, 'accountId':accountId})
    accountList = accountList.append(b, ignore_index=True )

len(transactions_response['accounts'])
# print(accountList)



def accountName(name):
    result = None
#     result = 'debit' if name=='1XxA8g36Dks69YKZVM3Yfbgpprwo7ECmVv9zw' else None
    if result == None:
        result= accountList[accountList['accountId']==name]['account'].values[0]
    if (result == "Visa Cad"):
        result = "Visa"
    elif (result == 'Personal Line Of Credit'):
        result = "Line"
    elif (result=="Eadvantage Savings Account"):
        result = "Savings"
    elif (result=="Tfsa Tax Advantage Savings Account"):
        result= "TFSA"
    elif (result== "Personal Chequing Account USD"):
        result = "USD"
    elif (result=="CIBC Smart Account"):
        result = "Debit"
        
    return result
# print(accountName('qb9a5qN6VKsXbjd5qYejHa6rrnoAvLhJjYgmM'))




In [29]:
access_token = 'access-development-046badf0-ded1-4c12-8c14-533038e5f363'

In [30]:
def getTrans(start_date, end_date):
    global access_token
    response = client.Transactions.get(access_token,
                                       start_date=start_date,
                                       end_date=end_date, count=500)
    print("# Transactions found" , response['total_transactions'])
    transactions = response['transactions']

    # Manipulate the count and offset parameters to paginate
    # transactions and retrieve all available data
    while len(transactions) < response['total_transactions']:
        response = client.Transactions.get(access_token,
                                           start_date='2018-01-01',
                                           end_date='2019-02-01',
                                           offset=len(transactions)
                                          )
        transactions.extend(response['transactions'])
    tx= pd.DataFrame(columns=['name', 'date', 'balance'])
    
    for account in transactions:
        name = account['name']
        date = account['date']
        amount = account['amount']
        accountId= accountName(account['account_id'])

        #print(name)
        #print(date)
        #print(amount)
        #print(account['category'])
        #print(account['category_id'])
        #print('\n------------------ \n')

        b = pd.Series({'name':name,'date':date,'balance':amount, 'accountId':accountId})
        tx = tx.append(b, ignore_index=True )
    return tx

In [31]:
tx = getTrans('2019-01-01', '2019-01-31')

# Transactions found 100


In [32]:
tx['accountId'].unique()


array(['Visa', 'USD', 'Debit'], dtype=object)